In [1]:
import sys
import os

# 基本信息
print(os.getcwd())
print(sys.path)

/home/ubuntu/Fed_Expr/MyExpr/notebook
['/home/ubuntu/Fed_Expr/MyExpr/notebook', '/home/ubuntu/miniconda/envs/fedml/lib/python37.zip', '/home/ubuntu/miniconda/envs/fedml/lib/python3.7', '/home/ubuntu/miniconda/envs/fedml/lib/python3.7/lib-dynload', '', '/home/ubuntu/miniconda/envs/fedml/lib/python3.7/site-packages', '/home/ubuntu/miniconda/envs/fedml/lib/python3.7/site-packages/IPython/extensions', '/home/ubuntu/.ipython']


In [34]:
import csv

import numpy as np
from sklearn.cluster import KMeans


class DataLoader(object):
    def __init__(self, data_name, data_path, client_list, sample_num_in_total, beta):
        # SUSY, Room Occupancy;
        self.data_name = data_name
        self.data_path = data_path
        self.client_list = client_list
        self.sample_num_in_total = sample_num_in_total
        self.beta = beta
        self.streaming_full_dataset_X = []
        self.streaming_full_dataset_Y = []
        self.StreamingDataDict = {}

    """
        return streaming_data
            key: client_id
            value: [sample1, sample2, ..., sampleN]
                    sample: {"x": [1,2,3,4,5,...,M]; "y":0}
    """

    def load_datastream(self):
        # 1 put sample_num_in_total data in streaming_full_dataset_X and streaming_full_dataset_Y
        self.preprocessing()
        # print(self.streaming_full_dataset_X)
        # print(self.streaming_full_dataset_Y)
        
        # 2 Kmeans cluster and assaign each client a cluster of data only sample_num_in_total*beta data are used
        self.load_adversarial_data()
        
        # 3 （1）Kmeans聚类可能不均衡，先砍掉超出的，随机补上
        #   （2）把剩余的 sample_num_in_total*（1 - beta）个数据补入每个类中
        self.load_stochastic_data()
        # for value in self.StreamingDataDict.values():
        #     random.shuffle(value)

        # for client_index in self.client_list:
        #     length = len(self.StreamingDataDict[client_index])
        #     logging.info("len of index %d = %d" % (client_index, length))
        return self.StreamingDataDict

    # beta (clustering, GMM)
    def load_adversarial_data(self):
        streaming_data = self.read_csv_file_for_cluster(self.beta)
        return streaming_data

    def load_stochastic_data(self):
        streaming_data = self.read_csv_file(self.beta)
        return streaming_data

    def read_csv_file(self, percent):

        # print("start from:")
        iteration_number = int(self.sample_num_in_total / len(self.client_list))
        index_start = int(percent * self.sample_num_in_total)
        stochastic_data_x = []
        stochastic_data_y = []
        # 将 streaming_full_dataset 中大于 percent * self.sample_num_in_total 的部分放入 stochastic_data
        for i_x, dp_x in enumerate(self.streaming_full_dataset_X):
            if i_x >= index_start:
                stochastic_data_x.append(dp_x)
        for i_y, dp_y in enumerate(self.streaming_full_dataset_Y):
            if i_y >= index_start:
                stochastic_data_y.append(dp_y)
        # 所有client中，取自身数据量大于均值的，surplus 放入 stochastic_data，并在它们的本地数据中去掉这部分数据
        for c_index in self.client_list:
            if len(self.StreamingDataDict[self.client_list[c_index]]) > iteration_number:
                for i, data_point in enumerate(self.StreamingDataDict[self.client_list[c_index]]):
                    if i >= iteration_number:
                        stochastic_data_x.append(data_point['x'])
                        stochastic_data_y.append(data_point['y'])
                self.StreamingDataDict[self.client_list[c_index]] = self.StreamingDataDict[self.client_list[c_index]][0:iteration_number]

        # print("***")
        # for c_index in self.client_list:
        #     print(len(self.StreamingDataDict[self.client_list[c_index]]))
        # print("***")
        client_index = 0
        full_count = 0
        # print("iteration_number = " + str(iteration_number))
        # print("len stochastic_data_x = " + str(len(stochastic_data_x)))
        for i in range(len(stochastic_data_x)):
            while len(self.StreamingDataDict[self.client_list[client_index]]) == iteration_number:
                client_index += 1
                full_count += 1
            sample = {}
            sample["x"] = stochastic_data_x[i]
            sample["y"] = stochastic_data_y[i]
            self.StreamingDataDict[self.client_list[client_index]].append(sample)
            
            # 估摸着是为了防止越界，写得太臭了
            if len(self.StreamingDataDict[self.client_list[client_index]]) == iteration_number and \
                    full_count == len(self.client_list) - 1:
                full_count += 1
            if full_count == len(self.client_list):
                # print("stop at index = " + str(i))
                break
        return self.StreamingDataDict

    def read_csv_file_for_cluster(self, percent):
        data = []
        label = []
        for client_id in self.client_list:
            self.StreamingDataDict[client_id] = []
        if percent == 0:
            return self.StreamingDataDict

        for i, row in enumerate(self.streaming_full_dataset_X):
            if i >= (self.sample_num_in_total * percent):
                break
            data.append(self.streaming_full_dataset_X[i])
            label.append(self.streaming_full_dataset_Y[i])
        
        # self.sample_num_in_total * percent条数据
        # print(len(data))
        # print(len(label))
        
        # print("Clustering Started")
        clusters = self.kMeans(data)
        # print("Clustering Finished")
        
        # 将data分类，cluster[i]表示data[i]的分类
        # print(clusters)
        # print(type(clusters))

        for i, cluster in enumerate(clusters):
            sample = {}
            sample["y"] = label[i]
            sample["x"] = data[i]
            
            # 将cluster分类放入对应的客户数据集中
            self.StreamingDataDict[self.client_list[cluster]].append(sample)
        # print("Arrange Clustered Data has Finished")

        # for id in self.client_list:
            # print("after clustering:")
            # print(len(self.StreamingDataDict[self.client_list[id]]))
        return self.StreamingDataDict

    def kMeans(self, X):
        kmeans = KMeans(n_clusters=len(self.client_list))
        kmeans.fit(X)
        return kmeans.labels_

    def preprocessing(self):
        # print("sample_num_in_total = " + str(self.sample_num_in_total))
        data = []
        with open(self.data_path) as csvfile:
            readCSV = csv.reader(csvfile, delimiter=",")
            for i, row in enumerate(readCSV):
                if i < self.sample_num_in_total:
                    if self.data_name == "SUSY":
                        data.append(np.asarray(row[1:], dtype=np.float32))
                        self.streaming_full_dataset_Y.append(int(row[0].split('.')[0]))
                    elif self.data_name == "RO":
                        data.append(np.asarray(row[2:-1], dtype=np.float32))
                        self.streaming_full_dataset_Y.append(int(row[-1].split('.')[0]))
                else:
                    break
        # min_max_scaler = preprocessing.MinMaxScaler()
        # self.streaming_full_dataset_X = min_max_scaler.fit_transform(data)
        self.streaming_full_dataset_X = data
        # print("############")
        # print(len(self.streaming_full_dataset_X))
        # print(self.streaming_full_dataset_X)

In [35]:
data_loader = DataLoader("SUSY", "../../FedML/data/UCI/SUSY/SUSY.csv", client_id_list, 5 * 10, 0.8)
streaming_data = data_loader.load_datastream()

[0 2 3 3 1 4 1 4 3 0 1 0 1 1 3 1 4 3 4 0 1 0 3 0 2 0 1 0 3 0 1 0 2 4 4 3 1
 3 4 3]
<class 'numpy.ndarray'>


In [ ]:
# 分析一下SUSY数据
client_id_list = [i for i in range(5)]

with open("../../FedML/data/UCI/SUSY/SUSY.csv") as csvfile:
    readCSV = csv.reader(csvfile, delimiter=",")
    # 查看前10条
    show = 10
    total = 0
    for row in readCSV:
        # total += 1
        # row[0]是类别（二分类，0/1），row[1:]为属性，总共18个属性
        if show > 0:
            print(len(row))
            print(len(np.asarray(row[1:], dtype=np.float32)))
            # 原类别是list
            print(type(row))
            # 改为numpy.ndarray
            # print(type(np.asarray(row[1:], dtype=np.float32)))
            print(row)
            print(np.asarray(row[1:], dtype=np.float32))
            break
        show -= 1
# len(SUSY) = 5,000,000
# The last 500,000 examples are used as a test set.n about your data set.
# print("total=", total)

In [39]:
# 简单观察一下dataloader对于batchsize的设计
from torch.utils.data import DataLoader


19
18
<class 'list'>
['0.000000000000000000e+00', '9.728614687919616699e-01', '6.538545489311218262e-01', '1.176224589347839355e+00', '1.157156467437744141e+00', '-1.739873170852661133e+00', '-8.743090629577636719e-01', '5.677649974822998047e-01', '-1.750000417232513428e-01', '8.100607395172119141e-01', '-2.525521218776702881e-01', '1.921887040138244629e+00', '8.896374106407165527e-01', '4.107718467712402344e-01', '1.145620822906494141e+00', '1.932632088661193848e+00', '9.944640994071960449e-01', '1.367815494537353516e+00', '4.071449860930442810e-02']
[ 0.97286147  0.65385455  1.1762246   1.1571565  -1.7398732  -0.87430906
  0.567765   -0.17500004  0.81006074 -0.25255212  1.921887    0.8896374
  0.41077185  1.1456208   1.9326321   0.9944641   1.3678155   0.0407145 ]
19
18
<class 'list'>
['1.000000000000000000e+00', '1.667973041534423828e+00', '6.419061869382858276e-02', '-1.225171446800231934e+00', '5.061022043228149414e-01', '-3.389389812946319580e-01', '1.672542810440063477e+00', '3.